# AP(P)3RO PROJECT (model de démo) #


#### M. Le Gras, A. Naullet, A. Calixte, P. Beaunieux ####

## Partie 1: Cartographie de la ville par le drone ##

On simule une ville avec un petit graphe

In [10]:
import numpy as np

In [45]:
#voici un graph simulant un morceau de ville
#(node1,node2, distance)                                                                                                                                 
G = [(1,2,29), (2,3,13), (3,4,21), (4,8,10), (8,2,2), (8,5,20), (8,6,12), (6,7,6), (6,1,22)]
print(G)

[(1, 2, 29), (2, 3, 13), (3, 4, 21), (4, 8, 10), (8, 2, 2), (8, 5, 20), (8, 6, 12), (6, 7, 6), (6, 1, 22)]


<img src="assets/img/graph_num.png" width="400">

### 1.2 Calcul du parcours le plus efficace pour le drone ###

L'idée c'est de trouver un chemin eulerien, cependant notre graphe n'est pas eulerien
Nous allons alors utilisé un algo qui transforme un graph non-eulerien en un graph eulerien (sans altéré ses propriétés). Enfin on pourra utiliser un algo qui trouve un chemin eulerien c-a-d un chemin qui passe par toutes les arretes du graphes (simulant les rues de la ville)

In [41]:
#ICI on fait l'algo du cycle_eulerien_avec_erreur

In [44]:
#fix le graph vers un graph eulerien
def odd_vertices(n, edges):
    deg = [0] * n
    for (a,b,c) in edges:
        deg[a] += 1
        deg[b] += 1
        
    return [a for a in range(n) if deg[a] % 2]

def is_edges(n, edges, node1, node2):
    for (a, b, c) in edges:
        if (a == node1 and b == node2) or (a == node2 and b == node1):
            return (a,b,c)
    return None

def list_edges(n, edges, l_vodd):
    res = []
    for a in l_vodd:
        for b in l_vodd:
            if (a == b):
                continue
            tmp = is_edges(n, edges, a, b)
            if (tmp and tmp not in res) :
                res.append(tmp)               
    return res

def shortest_edge_idx(l_edges):
    if (len(l_edges) == 0):
        return None
    shortest = l_edges[0][2]
    shortest_index = 0
    
    index = 1
    for (a,b,c) in l_edges[1:]:
        if (c < shortest):
            shortest = c
            shortest_index = index
        index+=1
    return l_edges[shortest_index]

def transform_to_eulerian(n, edges):
    list_vodd = odd_vertices(n, edges)
    l_edges = list_edges(n, edges, list_vodd)
    
    len_vodd = len(list_vodd)
    if (len_vodd == 2 or len_vodd == 0): #case is already eulerian
        return edges
    #print(list_vodd)
    #print(l_edges)
    while(len(list_vodd) != 2):
        shortest_edge = shortest_edge_idx(l_edges)
        #add edge between two
        (a,b,c) = shortest_edge
        new_edge = (b,a,c)
        edges.append(new_edge)
        
        #delete from vodd
        list_vodd.remove(a)
        list_vodd.remove(b)
        
        #delete from list_edges
        l_edges.remove(shortest_edge)
    return edges

In [6]:
# L'algo de transformation en graph eulerien doit donner le graph ci-dessous

<img src="assets/img/eulerian_graph.png" width="400">

In [36]:
def odd_(n, edges):
    deg = [0] * n
    for (a,b) in edges:
        deg[a] += 1
        deg[b] += 1
    return [a for a in range(n) if deg[a] % 2]

def is_edge_connected(n, edges):
    if n == 0 or len(edges) == 0:
        return True
    # Convert to adjacency list
    succ = [[] for a in range(n)]
    for (a,b) in edges:
        succ[a].append(b)
        succ[b].append(a)
    # BFS over the graph, starting from one extremity of the first edge
    touched = [False] * n
    init = edges[0][0]
    touched[init] = True
    todo = [init]
    while todo:
        s = todo.pop()
        for d in succ[s]:
            if touched[d]:
                continue
            touched[d] = True
            todo.append(d)
    for a in range(n):
        if succ[a] and not touched[a]:
            return False
    return True

def is_eulerian(n, edges):
    print(is_edge_connected(n, edges))
    print(not odd_(n, edges))
    return is_edge_connected(n, edges) and not odd_(n, edges)

def find_eulerian_cycle(n, edges):
    assert is_eulerian(n, edges)
    if len(edges) == 0:
        return []
    cycle = [edges[0][0]] # start somewhere
    while True:
        rest = []
        for (a, b) in edges:
            if cycle[-1] == a:
                cycle.append(b)
            elif cycle[-1] == b:
                cycle.append(a)
            else:
                rest.append((a,b))
        if not rest:
            assert cycle[0] == cycle[-1]
            return cycle[0:-1]
        edges = rest
        if cycle[0] == cycle[-1]:
            # Rotate the cycle so that the last state
            # has some outgoing edge in EDGES.
            for (a, b) in edges:
                if a in cycle:
                    idx = cycle.index(a)
                    cycle = cycle[idx:-1] + cycle[0:idx+1]
                    break


In [37]:
G = [(1,2,29), (2,3,13), (3,4,21), (4,8,10), (8,2,2), (8,5,20), (8,6,12), (6,7,6), (6,1,22)]   
print(G)
G = transform_to_eulerian(len(G), G)
G2 = []
for (a,b,c) in G:
    G2.append((a,b))
print(G)
print(G2)

# il manque les arretes (5,8) et (2,8) pour en faire un graph eulerien
# fix l'algo transform to eulerian pour obtenir le bon graph eulerien

G2.append((5,8))
G2.append((2,8))
print(G2)
print(find_eulerian_cycle(len(G2), G2))

[(1, 2, 29), (2, 3, 13), (3, 4, 21), (4, 8, 10), (8, 2, 2), (8, 5, 20), (8, 6, 12), (6, 7, 6), (6, 1, 22)]
[(1, 2, 29), (2, 3, 13), (3, 4, 21), (4, 8, 10), (8, 2, 2), (8, 5, 20), (8, 6, 12), (6, 7, 6), (6, 1, 22), (7, 6, 6)]
[(1, 2), (2, 3), (3, 4), (4, 8), (8, 2), (8, 5), (8, 6), (6, 7), (6, 1), (7, 6)]
[(1, 2), (2, 3), (3, 4), (4, 8), (8, 2), (8, 5), (8, 6), (6, 7), (6, 1), (7, 6), (5, 8), (2, 8)]
True
True
[1, 2, 3, 4, 8, 2, 8, 5, 8, 6, 7, 6]


In [6]:
# ICI ON AFFICHE le parcours

### 1.3 Recuperation des données du drones (simulation d'eneigement) ###

In [7]:
#ICI on simule l'eneigement (baill de plustard)